# Connect to Drive

# Preparing the data

In [1]:
import numpy as np
import pandas as pd
from typing import Callable, Optional, List, Tuple
from dataclasses import dataclass


@dataclass
class Filter:
    name: str
    func: Callable[[pd.DataFrame], pd.DataFrame]



@dataclass
class Feature:
    name: str
    func: Callable[[pd.DataFrame], float]



class Dataset(object):
    def __init__(self, name: str, path: str, filters: List[Filter], attack_split_factor: int = 5000):
        self._name = name
        self._path = path
        self._dataset_filters = filters
        self._attack_split_factor = attack_split_factor
        self._data = None


    def _load(self) -> None:
        df = pd.read_csv(self._path)

        print(f"Loaded dataset from '{self._path}'")

        # Filter the dataset if necessary.
        for f in self._dataset_filters:
            print(f"Filtering dataset with with '{f.name=}'")
            df = f.func(df)

        # Find the start of all attack blocks.
        block_starts = df.Time.diff().fillna(self._attack_split_factor) // self._attack_split_factor
        block_starts = block_starts[block_starts > 0]

        # Group all messages of the same block.
        blocks = pd.DataFrame(block_starts).reset_index().reset_index().drop(columns=["Time"]).rename(columns={"index": "Index", "level_0": "Block"})
        df = pd.merge_asof(df, blocks, on='Index')
        self._data = df.groupby("Block")

        # Make sure each block has only one attack label.
        min_label = self._data.apply(lambda x: x.Label.min())
        max_label = self._data.apply(lambda x: x.Label.max())
        assert min_label.to_list() == max_label.to_list(), "Multiple attack types in the same block!"


    def get_data(self) -> pd.DataFrame:
        if self._data is None:
            self._load()
        return self._data


    def create_feature_set(self, *, filters: List[Filter] = [], features: List[Feature] = [], remove_class: int = None, one_hot_label: bool = False, fillna_value: float = 0.0) -> Tuple[np.ndarray, np.array]:
        df = self.get_data()
        frames = []

        if remove_class is not None:
          df = df.apply(lambda block: block if block.Label.min() != remove_class else None).reset_index(level=0, drop=True).groupby("Block")


        # Filter the dataset if necessary.
        for i, f in enumerate(filters):
            print(f"Filtering with '{f.name=}' ({i+1} / {len(filters)})")
            df = df.apply(f.func).reset_index(level=0, drop=True).groupby("Block")

        # Calculate the features.
        for i, feat in enumerate(features):
            print(f"Calculating '{feat.name=}' ({i+1} / {len(features)})")
            frames.append(pd.DataFrame({feat.name: df.apply(feat.func)}))

        # Get the min label since we know it's unique for each block.
        y = df.apply(lambda block: block.Label.min()).to_list()
        # Encode the label into one-hot if necessary.
        if one_hot_label:
            y = [Dataset.to_onehot(label) for label in y]

        # Convert the dataset into numpy arrays for training/inference.
        return pd.concat(frames, axis=1).fillna(fillna_value).to_numpy(), np.array(y)



    @staticmethod
    def to_onehot(label: int) -> List[int]:
        return [(1 if i == label else 0) for i in range(5)]


    @staticmethod
    def from_onehot(label: List[int], approximate=False) -> int:
        if not approximate:
            return label.index(1)
        else:
            return np.argmax(label)

## Features to use

In [2]:
# Features we want to use for training/inference.

default_features = [
    Feature(name="signal1_stdev", func=lambda x: (x.Signal1_of_ID.std() if not (x.Signal1_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal2_stdev", func=lambda x: (x.Signal2_of_ID.std() if not (x.Signal2_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal3_stdev", func=lambda x: (x.Signal3_of_ID.std() if not (x.Signal3_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal4_stdev", func=lambda x: (x.Signal4_of_ID.std() if not (x.Signal4_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal1_mean", func=lambda x: (x.Signal1_of_ID.mean() if not (x.Signal1_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal2_mean", func=lambda x: (x.Signal2_of_ID.mean() if not (x.Signal2_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal3_mean", func=lambda x: (x.Signal3_of_ID.mean() if not (x.Signal3_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal4_mean", func=lambda x: (x.Signal4_of_ID.mean() if not (x.Signal4_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal1_median", func=lambda x: (x.Signal1_of_ID.median() if not (x.Signal1_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal2_median", func=lambda x: (x.Signal2_of_ID.median() if not (x.Signal2_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal3_median", func=lambda x: (x.Signal3_of_ID.median() if not (x.Signal3_of_ID.isnull().all()) else 0.0)),
    Feature(name="signal4_median", func=lambda x: (x.Signal4_of_ID.median() if not (x.Signal4_of_ID.isnull().all()) else 0.0)),
    Feature(name="message_count", func=lambda x: len(x.index)),
]

car_hacking_default_features = [
    Feature(name="signal0_stdev", func=lambda x: (x.DATA0.std() if not (x.DATA0.isnull().all()) else 0.0)),
    Feature(name="signal1_stdev", func=lambda x: (x.DATA1.std() if not (x.DATA1.isnull().all()) else 0.0)),
    Feature(name="signal2_stdev", func=lambda x: (x.DATA2.std() if not (x.DATA2.isnull().all()) else 0.0)),
    Feature(name="signal3_stdev", func=lambda x: (x.DATA3.std() if not (x.DATA3.isnull().all()) else 0.0)),
    Feature(name="signal4_stdev", func=lambda x: (x.DATA4.std() if not (x.DATA4.isnull().all()) else 0.0)),
    Feature(name="signal5_stdev", func=lambda x: (x.DATA5.std() if not (x.DATA5.isnull().all()) else 0.0)),
    Feature(name="signal6_stdev", func=lambda x: (x.DATA6.std() if not (x.DATA6.isnull().all()) else 0.0)),
    Feature(name="signal7_stdev", func=lambda x: (x.DATA7.std() if not (x.DATA7.isnull().all()) else 0.0)),
    Feature(name="signal0_mean", func=lambda x: (x.DATA0.mean() if not (x.DATA0.isnull().all()) else 0.0)),
    Feature(name="signal1_mean", func=lambda x: (x.DATA1.mean() if not (x.DATA1.isnull().all()) else 0.0)),
    Feature(name="signal2_mean", func=lambda x: (x.DATA2.mean() if not (x.DATA2.isnull().all()) else 0.0)),
    Feature(name="signal3_mean", func=lambda x: (x.DATA3.mean() if not (x.DATA3.isnull().all()) else 0.0)),
    Feature(name="signal4_mean", func=lambda x: (x.DATA4.mean() if not (x.DATA4.isnull().all()) else 0.0)),
    Feature(name="signal5_mean", func=lambda x: (x.DATA5.mean() if not (x.DATA5.isnull().all()) else 0.0)),
    Feature(name="signal6_mean", func=lambda x: (x.DATA6.mean() if not (x.DATA6.isnull().all()) else 0.0)),
    Feature(name="signal7_mean", func=lambda x: (x.DATA7.mean() if not (x.DATA7.isnull().all()) else 0.0)),
    Feature(name="signal0_median", func=lambda x: (x.DATA0.median() if not (x.DATA0.isnull().all()) else 0.0)),
    Feature(name="signal1_median", func=lambda x: (x.DATA1.median() if not (x.DATA1.isnull().all()) else 0.0)),
    Feature(name="signal2_median", func=lambda x: (x.DATA2.median() if not (x.DATA2.isnull().all()) else 0.0)),
    Feature(name="signal3_median", func=lambda x: (x.DATA3.median() if not (x.DATA3.isnull().all()) else 0.0)),
    Feature(name="signal4_median", func=lambda x: (x.DATA4.median() if not (x.DATA4.isnull().all()) else 0.0)),
    Feature(name="signal5_median", func=lambda x: (x.DATA5.median() if not (x.DATA5.isnull().all()) else 0.0)),
    Feature(name="signal6_median", func=lambda x: (x.DATA6.median() if not (x.DATA6.isnull().all()) else 0.0)),
    Feature(name="signal7_median", func=lambda x: (x.DATA7.median() if not (x.DATA7.isnull().all()) else 0.0)),
    Feature(name="message_count", func=lambda x: len(x.index)),
]

## Filters to use

In [3]:
# Filters for the feature set.

# Filter by ID.
id1_only = Filter(name="only_id1", func=lambda x: x[x.ID == "id1"])
id2_only = Filter(name="only_id2", func=lambda x: x[x.ID == "id2"])
id3_only = Filter(name="only_id3", func=lambda x: x[x.ID == "id3"])
id4_only = Filter(name="only_id4", func=lambda x: x[x.ID == "id4"])

# Filter by time.
start_50ms = Filter(name="start50ms", func=lambda x: x[x.Time <= (50 + x.Time.min())])
start_100ms = Filter(name="start100ms", func=lambda x: x[x.Time <= (100 + x.Time.min())])
start_200ms = Filter(name="start200ms", func=lambda x: x[x.Time <= (200 + x.Time.min())])
start_500ms = Filter(name="start500ms", func=lambda x: x[x.Time <= (500 + x.Time.min())])
start_1000ms = Filter(name="start1000ms", func=lambda x: x[x.Time <= (1000 + x.Time.min())])

## Datasets created

In [4]:
# Pre-filtered version of the 'AllData'; contains only attack messages, and attack labels go from 0 to 4.
# You should get the same data if you use this one or the AllData with the two default filters above.
AttackOnly = Dataset(
    name="AllData",
    path="/kaggle/input/attackonly/attack_only.csv",
    filters=[],
    attack_split_factor=1_000
)


CarHackingAttacks = Dataset(
    name="AllData",
    path="/kaggle/input/attack-only/car-hacking_attack_only.csv",
    filters=[],
    attack_split_factor=5_00
)



# Load data to use

In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler

class CANBusFewShotDataset(Dataset):
    def __init__(self, X, y):
        """
        Convert CAN Bus feature vectors into image-like format.

        Args:
        - X: NumPy array of shape (num_samples, num_features)
        - y: NumPy array of shape (num_samples, ) (labels)
        """
        # Get the number of signals dynamically
        num_signals = (X.shape[1] - 1) // 3  # -1 for message_count
        
        # Reshape to (num_samples, num_signals, 3)
        num_samples = X.shape[0]

        # Rearrange the features to have stdev, mean, median in columns
        # Original order: stdev(0-n), mean(0-n), median(0-n), message_count
        # Desired order: stdev(0-n), mean(0-n), median(0-n) as columns for each signal

        # Create index ranges for each feature type
        stdev_indices = list(range(num_signals))
        mean_indices = list(range(num_signals, 2 * num_signals))
        median_indices = list(range(2 * num_signals, 3 * num_signals))

        # Create a new array with the desired shape and order
        reshaped_features = np.zeros((num_samples, num_signals, 3))

        for i in range(num_signals):  # For each signal
            reshaped_features[:, i, 0] = X[:, stdev_indices[i]]
            reshaped_features[:, i, 1] = X[:, mean_indices[i]]
            reshaped_features[:, i, 2] = X[:, median_indices[i]]

        self.features = torch.tensor(reshaped_features, dtype=torch.float32)

        # Scale features to the range [0, 1] using MinMaxScaler
        scaler = MinMaxScaler()
        self.features = torch.tensor(scaler.fit_transform(self.features.reshape(-1, 1)).reshape(-1, num_signals, 3), dtype=torch.float32)

        # Expand to 3 channels (simulate RGB) - NOW with the correct shape
        self.features = self.features.unsqueeze(1).repeat(1, 3, 1, 1)  # Shape: (batch, 3, num_signals, 3)

        self.labels = torch.tensor(y, dtype=torch.long)  # Convert labels to tensor

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def get_labels(self):
        return self.labels.tolist()


In [6]:
# Load `car_hacking_default_features` dataset
X, y = CarHackingAttacks.create_feature_set(
    filters=[],  # No additional filters
    features=car_hacking_default_features,
)
X2, y2 = AttackOnly.create_feature_set(
    filters=[],  # No additional filters
    features=default_features,
)


# Split into train and test
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.2, random_state=0, shuffle=True)


# Convert to Few-Shot Learning Dataset Format
carhacking_dataset = CANBusFewShotDataset(X_train, y_train)
val_ch_dataset = CANBusFewShotDataset(X_val, y_val)
attack_dataset = CANBusFewShotDataset(X2_train, y2_train)
val_att_dataset = CANBusFewShotDataset(X2_val, y2_val)


# Check new dataset format
print(f"Dataset size: {len(carhacking_dataset)}")
print(f"Example data point shape: {carhacking_dataset[0][0].shape}")  
print(f"Dataset size: {len(attack_dataset)}")
print(f"Example data point shape: {attack_dataset[0][0].shape}")  

<ipython-input-1-3255faebca60>:31: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self._path)


Loaded dataset from '/kaggle/input/attack-only/car-hacking_attack_only.csv'


<ipython-input-1-3255faebca60>:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  min_label = self._data.apply(lambda x: x.Label.min())
<ipython-input-1-3255faebca60>:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_label = self._data.apply(lambda x: x.Label.max())


Calculating 'feat.name='signal0_stdev'' (1 / 25)


<ipython-input-1-3255faebca60>:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  frames.append(pd.DataFrame({feat.name: df.apply(feat.func)}))


...

<ipython-input-1-3255faebca60>:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  min_label = self._data.apply(lambda x: x.Label.min())
<ipython-input-1-3255faebca60>:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  max_label = self._data.apply(lambda x: x.Label.max())


Calculating 'feat.name='signal1_stdev'' (1 / 13)
Calculating 'feat.name='signal2_stdev'' (2 / 13)


<ipython-input-1-3255faebca60>:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  frames.append(pd.DataFrame({feat.name: df.apply(feat.func)}))


Calculating 'feat.name='signal3_stdev'' (3 / 13)
Calculating 'feat.name='signal4_stdev'' (4 / 13)
Calculating 'feat.name='signal1_mean'' (5 / 13)
Calculating 'feat.name='signal2_mean'' (6 / 13)
Calculating 'feat.name='signal3_mean'' (7 / 13)
Calculating 'feat.name='signal4_mean'' (8 / 13)
Calculating 'feat.name='signal1_median'' (9 / 13)
Calculating 'feat.name='signal2_median'' (10 / 13)
Calculating 'feat.name='signal3_median'' (11 / 13)
Calculating 'feat.name='signal4_median'' (12 / 13)
Calculating 'feat.name='message_count'' (13 / 13)
Dataset size: 962
Example data point shape: torch.Size([3, 8, 3])
Dataset size: 436
Example data point shape: torch.Size([3, 4, 3])


<ipython-input-1-3255faebca60>:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  y = df.apply(lambda block: block.Label.min()).to_list()


# Train a model with Episodic Training
Download FSL.

In [7]:
try:
    import google.colab
    colab = True
except:
    colab = False

if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1201, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 1201 (delta 250), reused 180 (delta 176), pack-reused 841 (from 1)
Receiving objects: 100% (1201/1201), 2.31 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (721/721), done.
/kaggle/working/easy-few-shot-learning
Processing /kaggle/working/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easyfsl: filename=easyfsl-1.5.0-py3-none-any.whl size=73036 sha256=3fa64e3933b73c1bcc8f968cbba824a8699101928c89773df73c43c5d0aec306
  Stored in directory: /root/.cache/pip/wheels/42/3c/be/1c46bdf231c4190667375ac0fc96270093e9693ea3ca24a322
Successfully built easyfsl


In [8]:
import copy
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

Then we're gonna do the most important thing in Machine Learning research: ensuring reproducibility by setting the random seed. We're going to set the seed for all random packages that we could possibly use, plus some other stuff to make CUDA deterministic (see [here](https://pytorch.org/docs/stable/notes/randomness.html)).

I strongly encourage that you do this in **all your scripts**.

In [9]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Then we're gonna set the shape of our problem.

Also we define our set-up, like the device (change it if you don't have CUDA) or the number of workers for data loading.

In [10]:
# Number of attack ids to evaluate to
n_way = 4

# Amount of support features per attack id model has to make a judgment 
n_shot = 5

# Number of features per attack id that model has to evaluate per task
n_query = 10

DEVICE = "cuda"

n_workers = 4

## Training

First we define our data loaders for training and validation. I set it up for the carhacking data and the attack data

In [11]:

from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 15
n_validation_tasks = 5


train_sampler = TaskSampler(carhacking_dataset, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch)
train_loader = DataLoader(carhacking_dataset, batch_sampler=train_sampler, num_workers=n_workers, pin_memory=True, collate_fn=train_sampler.episodic_collate_fn)

atk_train_sampler = TaskSampler(attack_dataset, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch)
atk_train_loader = DataLoader(attack_dataset, batch_sampler=atk_train_sampler, num_workers=n_workers, pin_memory=True, collate_fn=atk_train_sampler.episodic_collate_fn)

val_sampler = TaskSampler(val_ch_dataset, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks)
val_loader = DataLoader(val_ch_dataset, batch_sampler=val_sampler, num_workers=n_workers, pin_memory=True, collate_fn=val_sampler.episodic_collate_fn)

atk_val_sampler = TaskSampler(val_att_dataset, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks)
atk_val_loader = DataLoader(val_ch_dataset, batch_sampler=atk_val_sampler, num_workers=n_workers, pin_memory=True, collate_fn=atk_val_sampler.episodic_collate_fn)

And then we define the network. Here I chose Prototypical Networks and the built-in ResNet18 from PyTorch because it's easy.

In [12]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier
from easyfsl.modules import resnet12


convolutional_network = resnet12()
convolutional_network2 = resnet12()
few_shot_classifier = PrototypicalNetworks(convolutional_network).to(DEVICE)
attack_classifier = PrototypicalNetworks(convolutional_network2).to(DEVICE)

Now let's define our training helpers ! I chose to use Stochastic Gradient Descent on 200 epochs with a scheduler that divides the learning rate by 10 after 120 and 160 epochs. The strategy is derived from [this repo](https://github.com/fiveai/on-episodes-fsl).

We're also gonna use a TensorBoard because it's always good to see what your training curves look like.

In [13]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
atk_train_optimizer = SGD(
    attack_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)
atk_train_scheduler = MultiStepLR(
    atk_train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

And now let's get to it! Here we define the function that performs a training epoch.

We use tqdm to monitor the training in real time in our logs.

In [14]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

And we have everything we need! To perform validations we'll just use the built-in `evaluate` function from `easyfsl.methods.utils`.

This is now the time to **start training**.

I added something to log the state of the model that gave the best performance on the validation set.

In [15]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = copy.deepcopy(few_shot_classifier.state_dict())
        # state_dict() returns a reference to the still evolving model's state so we deepcopy
        # https://pytorch.org/tutorials/beginner/saving_loading_models
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|██████████| 5/5 [00:00<00:00, 34.17it/s, accuracy=1]

Ding ding ding! We found a new best model!
Epoch 1



Validation: 100%|██████████| 5/5 [00:00<00:00, 42.20it/s, accuracy=1]

Epoch 2


...


Validation: 100%|██████████| 5/5 [00:00<00:00, 48.72it/s, accuracy=1]

Epoch 198



Validation: 100%|██████████| 5/5 [00:00<00:00, 48.62it/s, accuracy=1]

Epoch 199



Validation: 100%|██████████| 5/5 [00:00<00:00, 52.11it/s, accuracy=1]


In [16]:
few_shot_classifier.load_state_dict(best_state)

<All keys matched successfully>

# ATTACK

In [17]:
# Attack


LOSS_FUNCTION = nn.CrossEntropyLoss()

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

best_state = attack_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(attack_classifier, atk_train_loader, atk_train_optimizer)
    validation_accuracy = evaluate(
        attack_classifier, atk_val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = copy.deepcopy(attack_classifier.state_dict())
        # state_dict() returns a reference to the still evolving model's state so we deepcopy
        # https://pytorch.org/tutorials/beginner/saving_loading_models
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    atk_train_scheduler.step()

Epoch 0


Validation: 100%|██████████| 5/5 [00:00<00:00, 51.35it/s, accuracy=1]

Ding ding ding! We found a new best model!
Epoch 1



Validation: 100%|██████████| 5/5 [00:00<00:00, 48.30it/s, accuracy=1]

Epoch 2


...


Validation: 100%|██████████| 5/5 [00:00<00:00, 50.27it/s, accuracy=1]

Epoch 198



Validation: 100%|██████████| 5/5 [00:00<00:00, 40.72it/s, accuracy=1]

Epoch 199



Validation: 100%|██████████| 5/5 [00:00<00:00, 53.08it/s, accuracy=1]


Congrats! You performed Episodic Training using EasyFSL. If you want to compare with a model trained using classical training, look at [this other example notebook](classical_training.ipynb).


# Classical Model

In [18]:
from torch.utils.data import DataLoader


batch_size = 128
classical_train_loader = DataLoader(carhacking_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

classical_classifier = resnet12(
    use_fc=True,
    num_classes=len(set(carhacking_dataset.get_labels())),
).to(DEVICE)

val_classifier = PrototypicalNetworks(classical_classifier).to(DEVICE)

LOSS_FUNCTION = nn.CrossEntropyLoss()

scheduler_milestones = [150, 180]
scheduler_gamma = 0.1
learning_rate = 1e-01
tb_logs_dir = Path(".")

classical_train_optimizer = SGD(
    classical_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
classical_train_scheduler = MultiStepLR(
    classical_train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))


In [19]:
def training_epoch2(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = LOSS_FUNCTION(model_(images.to(DEVICE)), labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [20]:
from easyfsl.utils import evaluate


best_state = classical_classifier.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch2(classical_classifier, classical_train_loader, classical_train_optimizer)

    if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
        classical_classifier.set_use_fc(False)
        validation_accuracy = evaluate(
            val_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
        )
        classical_classifier.set_use_fc(True)

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = copy.deepcopy(val_classifier.state_dict())
            # state_dict() returns a reference to the still evolving model's state so we deepcopy
            # https://pytorch.org/tutorials/beginner/saving_loading_models
            print("Ding ding ding! We found a new best model!")

        tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Training: 100%|██████████| 8/8 [00:00<00:00, 18.39it/s, loss=0.2]


Epoch 1


Training: 100%|██████████| 8/8 [00:00<00:00, 25.20it/s, loss=2.79e-5]


Epoch 2


...

Training: 100%|██████████| 8/8 [00:00<00:00, 25.02it/s, loss=1.37e-5]


Epoch 198


Training: 100%|██████████| 8/8 [00:00<00:00, 25.20it/s, loss=9.26e-6]


Epoch 199


Validation: 100%|██████████| 5/5 [00:00<00:00, 43.38it/s, accuracy=1]


In [21]:
from torch.utils.data import DataLoader

batch_size = 128
classical_train_loader = DataLoader(attack_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers)

classical_classifier = resnet12(
    use_fc=True,
    num_classes=len(set(attack_dataset.get_labels())),
).to(DEVICE)

val_classifier = PrototypicalNetworks(classical_classifier).to(DEVICE)

LOSS_FUNCTION = nn.CrossEntropyLoss()

scheduler_milestones = [150, 180]
scheduler_gamma = 0.1
learning_rate = 1e-01
tb_logs_dir = Path(".")

classical_train_optimizer = SGD(
    classical_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
classical_train_scheduler = MultiStepLR(
    classical_train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [22]:
from easyfsl.utils import evaluate


best_state = classical_classifier.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch2(classical_classifier, classical_train_loader, classical_train_optimizer)

    if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
        classical_classifier.set_use_fc(False)
        validation_accuracy = evaluate(
            val_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
        )
        classical_classifier.set_use_fc(True)

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = copy.deepcopy(val_classifier.state_dict())
            # state_dict() returns a reference to the still evolving model's state so we deepcopy
            # https://pytorch.org/tutorials/beginner/saving_loading_models
            print("Ding ding ding! We found a new best model!")

        tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Training: 100%|██████████| 4/4 [00:00<00:00,  9.63it/s, loss=1.91]


Epoch 1


Training: 100%|██████████| 4/4 [00:00<00:00, 13.54it/s, loss=2.37]


Epoch 2


...

Training: 100%|██████████| 4/4 [00:00<00:00, 12.86it/s, loss=0.115]


Epoch 198


Training: 100%|██████████| 4/4 [00:00<00:00, 13.20it/s, loss=0.0787]


Epoch 199


Validation: 100%|██████████| 5/5 [00:00<00:00, 51.63it/s, accuracy=1]
